In [5]:
# Importando Bibliotecas
import pandas as pd
import requests
import lxml.html as lh
import numpy as np


In [17]:
# URL da página web que contém a tabela de dados de doenças e sintomas
url = 'https://people.dbmi.columbia.edu/~friedma/Projects/DiseaseSymptomKB/index.html'

In [18]:
try:
    # Envia uma requisição GET para a URL e armazena a resposta na variável 'page'
    page = requests.get(url)
    # Verifica se a requisição foi bem-sucedida (status code 200)
    page.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")  # Exibe erro de HTTP
except requests.exceptions.ConnectionError as conn_err:
    print(f"Connection error occurred: {conn_err}")  # Exibe erro de conexão
except requests.exceptions.Timeout as timeout_err:
    print(f"Timeout error occurred: {timeout_err}")  # Exibe erro de timeout
except requests.exceptions.RequestException as req_err:
    print(f"An error occurred: {req_err}")  # Exibe erro genérico de requisição
else:
    # Se a requisição for bem-sucedida, converte o conteúdo da página em um objeto do tipo 'lxml'
    doc = lh.fromstring(page.content)
    # Utiliza XPath para encontrar todos os elementos <tr> (linhas de tabela) na página
    tr_elements = doc.xpath('//tr')

In [19]:
# Lista para armazenar os nomes das colunas e os dados de cada coluna
col = []

# Inicializa o contador para rastrear o índice das colunas
i = 0

# Itera sobre os elementos do cabeçalho da tabela (primeira linha)
for t in tr_elements[0]:
    # Incrementa o contador
    i += 1
    # Extrai o texto do elemento, remove quebras de linha e espaços extras
    name = str(t.text_content()).strip('\n').replace('\n', '').replace(' ', '')
    # Imprime o índice e o nome da coluna
    print('%d: "%s"' % (i, name))
    # Adiciona o nome da coluna e uma lista vazia para armazenar os dados dessa coluna
    col.append((name, []))


1: "Disease"
2: "CountofDiseaseOccurrence"
3: "Symptom"


In [20]:
def prewc(word):
    """
    Função para processar uma palavra, removendo o prefixo até o primeiro sublinhado, se existir.

    Parâmetros:
    word (str): A palavra a ser processada.

    Retorna:
    str: A palavra processada.
    """
    # Itera sobre cada caractere da palavra
    for i in range(len(word)):
        try:
            # Tenta converter a palavra inteira em um inteiro
            int(word)
            # Se a conversão for bem-sucedida, retorna a palavra original
            return word
        except:
            # Se a conversão falhar, verifica se o caractere atual é um sublinhado
            if word[i] == '_':
                # Se for um sublinhado, retorna a parte da palavra após o sublinhado
                return word[i + 1:]



In [21]:
def preoc(i: int):
    """
    Função para processar uma linha específica da tabela, extraindo e limpando os dados.

    Parâmetros:
    i (int): O índice da linha na tabela a ser processada.

    Retorna:
    list: Uma lista de palavras processadas da linha especificada.
    """
    # Extrai o texto da linha especificada, removendo quebras de linha e substituindo por vírgulas
    word = str(tr_elements[int(i)].text_content()
               .replace('\n  \n  \n  ', ',')
               .replace('\n  \n  ', '')
               .replace('\n  ', '')
               .replace('\n ', ''))
    
    # Divide o texto extraído em palavras, utilizando a vírgula como delimitador
    words = [prewc(w) for w in word.split(',')]
    
    # Retorna a lista de palavras processadas
    return words

In [10]:
# Inicializa um dicionário vazio para armazenar os dados
doc = {}
# Inicializa um contador
a = 0

# Itera sobre as linhas da tabela, começando da segunda linha (índice 1) até a última
for j in range(1, len(tr_elements)):
    # Processa a linha atual da tabela
    ans = preoc(j)
    # print(ans)  # Pode ser usado para depuração

    # Verifica se a primeira coluna da linha processada não é None
    if ans[0] is not None:
        # Incrementa o contador
        a += 1
        # Inicializa uma lista para armazenar os sintomas
        box = []
        # Adiciona o terceiro elemento da resposta (sintomas) na lista 'box'
        box.append(ans[2])
        # Adiciona uma nova entrada no dicionário 'doc' com a chave sendo o contador 'a' convertido para string
        doc[str(a)] = {
            'disease': ans[0],  # Doença
            'occurence': ans[1],  # Ocorrência
            'symptoms': box  # Sintomas
        }
    else:
        # Se a primeira coluna for None, adiciona o terceiro elemento da resposta na lista 'box'
        box.append(ans[2])

# Inicializa listas para armazenar dados de doenças, ocorrências, sintomas e número de sintomas
disease = []
occurrence = []
symptoms = []
symptomsno = []

# Itera sobre os valores no dicionário 'doc'
for c in doc.values():
    # Inicializa uma string para concatenar os sintomas
    sym = ''
    # Itera sobre a lista de sintomas e os concatena em uma única string separada por vírgulas
    for x in c['symptoms']:
        sym += ',' + str(x)
    # Adiciona os dados extraídos nas respectivas listas
    disease.append(c['disease'])  # Adiciona a doença à lista 'disease'
    symptomsno.append(len(c['symptoms']))  # Adiciona o número de sintomas à lista 'symptomsno'
    occurrence.append(int(c['occurence']))  # Adiciona a ocorrência à lista 'occurrence'
    symptoms.append(sym.strip(','))  # Adiciona a string de sintomas à lista 'symptoms', removendo a vírgula inicial


In [26]:
import pprint

# Inicializa um dicionário vazio para armazenar os dados
doc = {}
# Inicializa um contador
a = 0

# Itera sobre as linhas da tabela, começando da segunda linha (índice 1) até a última
for j in range(1, len(tr_elements)):
    # Processa a linha atual da tabela
    ans = preoc(j)
    # Verifica se a primeira coluna da linha processada não é None
    if ans[0] is not None:
        # Incrementa o contador
        a += 1
        # Inicializa uma lista para armazenar os sintomas
        box = []
        # Adiciona o terceiro elemento da resposta (sintomas) na lista 'box'
        box.append(ans[2])
        # Adiciona uma nova entrada no dicionário 'doc' com a chave sendo o contador 'a' convertido para string
        doc[str(a)] = {
            'disease': ans[0],  # Doença
            'occurence': ans[1],  # Ocorrência
            'symptoms': box  # Sintomas
        }
    else:
        # Se a primeira coluna for None, adiciona o terceiro elemento da resposta na lista 'box'
        box.append(ans[2])

# Visualiza o dicionário usando pprint
pprint.pprint(doc)


{'1': {'disease': 'hypertensivedisease',
       'occurence': '3363',
       'symptoms': ['painchest',
                    'shortnessof breath',
                    'dizziness',
                    'asthenia',
                    'fall',
                    'syncope',
                    'vertigo',
                    'sweat^UMLS:C0700590_sweatingincreased',
                    'palpitation',
                    'nausea',
                    'anginapectoris',
                    'pressurechest']},
 '10': {'disease': 'hypercholesterolemia',
        'occurence': '685',
        'symptoms': ['pain',
                     'pain chest',
                     'sweat^UMLS:C0700590_sweating increased',
                     'nonsmoker',
                     'pressure chest',
                     'syncope',
                     'numbness',
                     'chest discomfort',
                     'shortness of breath',
                     'st segment depression',
                     'worry',
 